In [1]:
import os
import shutil
import threading

import nglview as nv
import MDAnalysis as mda
import moviepy.editor as mpy

from MDAnalysis.tests.datafiles import PSF, DCD
from nglview.contrib.movie import MovieMaker
from time import sleep

### loading files

In [2]:
u1 = mda.Universe(PSF, DCD)

view = nv.show_mdanalysis(u1)
view

NGLWidget(max_frame=97)

## Object

In [3]:
# output folder
output_folder = "/home/yuyang/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook/test"

In [4]:
class MovieMaker:
    def __init__(self, view: nv.widget.NGLWidget, output: str = '.', filename: str = "sample") -> None:
        self.output = output
        self.tempfolder = f'{output}/_temp'
        self.view = view
        self.total_frame = 10
        self.frame_per_second = 3
        self.filename = filename
   
    def generate(self):
        # remove temp files
        self._remove_temp_files()

        # start a thread to generate images
        thread = threading.Thread(
            target = self._generate_images,
        )
        thread.daemon = True
        thread.start()

    def make(self): 
        # make sure to wait for the thread to finish
        # while thread.is_alive():
        #     if os.path.exists(f'{self.tempfolder}/0{self.filename}_{self.total_frame - 1}.png'):
        #         break
        #     else:
        #         sleep(0.5)
        
        # make gif
        self.total_frames = os.listdir(f'{output_folder}/_temp')
        self._make_gif()

        # remove temp files
        self._remove_temp_files()
  
    def _load_images(self):
        # get all (sorted) image files
        self.imagefiles = [f'{self.tempfolder}/0{self.filename}_{i}.png' for i in range(0, self.total_frame, 2)]

    # make a gif file
    def _make_gif(self):
        self._load_images()
        
        im = mpy.ImageSequenceClip(self.imagefiles, fps=self.frame_per_second)
        im.write_gif(f'{self.output}/{self.filename}.gif', fps=self.frame_per_second)

    def _remove_temp_files(self):
        if os.path.exists(self.tempfolder):
            shutil.rmtree(self.tempfolder)

    def _generate_images(self):
        images = [] # List[bytes]

        for frame in range(0, self.total_frame):
            # set frame to update coordinates
            self.view.frame = frame
            
            # make sure to let NGL spending enough time to update coordinates
            sleep(0.5)
            im1 = self.view.render_image()
            images.append(im1)

        for im in images: # wait for browser to finish rendering
            while not im.value:
                sleep(0.1)

        # check the output folder
        if not os.path.exists(f'{self.output}/_temp'):
            os.makedirs(self.tempfolder)

        for n, im in zip(range(0, self.total_frame), images):
            with open(f'{self.tempfolder}/0{self.filename}_{n}.png', 'wb') as fh:
                fh.write(im.value)
   

In [5]:
my_building_block = MovieMaker(view, output_folder)
my_building_block.generate()

In [7]:
my_building_block.make()


[MoviePy] Building file /home/yuyang/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook/test/sample.gif with imageio


 83%|████████▎ | 5/6 [00:00<00:00,  9.14it/s]


In [12]:
my_building_block.view.frame = 0
my_building_block.view.render_image()

Image(value=b'', width='99%')

## Display video in notebook

In [8]:
# display the gif in this notebook
from IPython import display

display.HTML(f"<img src='{output_folder}/sample.gif'></img>")